In [49]:
import pandas as pd 

import numpy as np

import matplotlib 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

                                                                                                                    Klasse zum Training eines DecissionTreeClassifiers:                                                                                                                                        
__init__ 
- Methode ist der Konstruktor der Klasse:
- alle Parameter mit "=" sind nicht verpflichtend, also ist es nur path zur csv-Datei                                                                                                                                                                                                                               
- ohne Angabe des Schlüsselwortes muss die Reihenfolge eingehalten werden, mit entsprechend nicht            
                                                                                                                                                                                                     
__data__:
- Methode kann man nicht von außen aufruen, das liegt an den __ quasi das private in Python     
                                                                                                                                                                                                               
--> Ziel ist es für jeden relevanten Parameter des DecissionTrees eine Methode zu bauen die diesen durchläuft. Diese Methoden sollen dann in einer train Methode kombiniert werden um alle Fälle abzudecken.

In [47]:
class myTree:
    def __init__(self, path, max_depthVal=0, max_depthMax=200, max_depthMin=0, min_sample_splitMax=5, min_sample_splitMin=2, clf=None):
        self.path = path
        self.trainList = self.__data()
        self.max_depthVal = max_depthVal
        self.max_depthMax = max_depthMax
        self.max_depthMin=max_depthMin
        self.min_sample_splitMax = min_sample_splitMax
        self.min_sample_splitMin = min_sample_splitMin
        self.clf = clf
    def __data(self):
        self.data = pd.read_csv(self.path)
        columnsLength = self.data.shape[1]-1
        X = self.data.iloc[:,0:columnsLength]
        y = self.data[["groupID"]].values.ravel()
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
        return [X_train, X_test, y_train, y_test]
    def out(self):
        if(self.clf == None):
            print("Modell ohne Parameter wird angelegt, da keines vorhanden:")
            self.clf = DecisionTreeClassifier(random_state=42)
            self.clf = self.clf.fit(self.trainList[0], self.trainList[2])
        predict = self.clf.predict(self.trainList[1])
        report = classification_report(self.trainList[3], predict, target_names=["Atheism", "Graphics", "Space", "Religion"])
        print(report)
        print("F-Score: ",f1_score(predict, self.trainList[3], average='macro'))
        print("Precision: ",precision_score(predict, self.trainList[3], average='macro'))
        print("Recall: ",recall_score(predict, self.trainList[3], average='macro'))
    def max_depth(self, min, max):
        self.max_depthMax = max
        self.max_depthMin = min
        f = 0
        for i in range(self.max_depthMax):
            if(i<=self.max_depthMin):
                continue
            clf = DecisionTreeClassifier(random_state=42, criterion="entropy", max_depth=i)
            clf = clf.fit(self.trainList[0], self.trainList[2])
            predict = clf.predict(self.trainList[1])
            if f1_score(predict, self.trainList[3], average='macro')>f:
                f = f1_score(predict, self.trainList[3], average='macro')
                self.max_depthVal = i
                self.clf =  DecisionTreeClassifier(random_state=42, criterion="entropy", max_depth=i)
                self.clf = self.clf.fit(self.trainList[0], self.trainList[2])      
        return self.max_depthVal
    def min_sample_split(self):
        pass


In [48]:
testTree = myTree("20newsgroups_processed.csv")
testTree.max_depth(100, 150)
testTree.out()

              precision    recall  f1-score   support

     Atheism       0.55      0.51      0.53       159
    Graphics       0.79      0.77      0.78       193
       Space       0.61      0.81      0.70       197
    Religion       0.54      0.35      0.43       125

    accuracy                           0.64       674
   macro avg       0.62      0.61      0.61       674
weighted avg       0.64      0.64      0.63       674

F-Score:  0.6081460258510376
Precision:  0.608844984871816
Recall:  0.6241808376421494


Visualisierung des Baumes: